In [7]:
import ipaddress
from behave import *
from guerrilla.connection import Session
from guerrilla.logging import logger
import time

In [8]:
config = {
    "protocol": "ssh",
    "host": "192.168.127.254",
    "username": "admin",
    "password": "moxa",
    "name": "EDR-G9010",
    "os_type": "edr_g9010"
}
device = Session(config)
device.connect()

2023-10-26 17:31:48.452 | INFO     | guerrilla.connection.ssh:__post_init__:23 - Created EDR-G9010's SSH session to 192.168.127.254:22
2023-10-26 17:31:48.453 | INFO     | guerrilla.connection.ssh:disconnect:44 - Closed connection to 192.168.127.254:22


2023-10-26 17:31:48.841 | SUCCESS  | guerrilla.connection.ssh:connect:35 - Connected to EDR-G9010 192.168.127.254:22


In [9]:
device.run('reload factory-default')
device.run('Y')
time.sleep(30)
device.connect()

2023-10-26 17:31:48.996 | DEBUG    | guerrilla.connection.ssh:run:78 - From EDR-G9010      sent command: reload factory-default
2023-10-26 17:31:49.098 | DEBUG    | guerrilla.connection.ssh:run:96 - From EDR-G9010  received  output: reload factory-default
Proceed with reload to factory default? [Y/n]
2023-10-26 17:31:49.099 | DEBUG    | guerrilla.connection.ssh:run:78 - From EDR-G9010      sent command: Y
2023-10-26 17:31:49.199 | DEBUG    | guerrilla.connection.ssh:run:96 - From EDR-G9010  received  output: Y
Firewall/VPN Router 00000# 
Firewall/VPN Router 00000#
2023-10-26 17:32:19.592 | SUCCESS  | guerrilla.connection.ssh:connect:35 - Connected to EDR-G9010 192.168.127.254:22


In [10]:
config = {
    "protocol": "ssh",
    "host": "192.168.121.88",
    "username": "root",
    "password": "vagrant",
    "name": "HOST A",
    "os_type": "linux"
}
host_a = Session(config)
host_a.connect()

dev = 'eth1'
ip = ['192.168.127.94']
mask = ['255.255.255.0']
# set networks
host_a.run(f'ip addr flush dev {dev}')
host_a.run(f'echo "source-directory /etc/network/interfaces.d" > /etc/network/interfaces')
config = f"auto {dev}\niface {dev} inet static\naddress {ip[0]}\nnetmask {mask[0]}"

for i in range(1, len(ip)):
        host_a.run(f'echo "up ip addr add {ip[i]}/{mask[i]} dev {dev}" >> /etc/network/interfaces.d/{dev}')

2023-10-26 17:32:19.899 | INFO     | guerrilla.connection.ssh:__post_init__:23 - Created HOST A's SSH session to 192.168.121.88:22
2023-10-26 17:32:19.900 | INFO     | guerrilla.connection.ssh:disconnect:44 - Closed connection to 192.168.121.88:22
2023-10-26 17:32:20.171 | SUCCESS  | guerrilla.connection.ssh:connect:35 - Connected to HOST A 192.168.121.88:22
2023-10-26 17:32:20.322 | DEBUG    | guerrilla.connection.ssh:run:78 - From HOST A         sent command: ip addr flush dev eth1
2023-10-26 17:32:20.423 | DEBUG    | guerrilla.connection.ssh:run:96 - From HOST A     received  output: ip addr flush dev eth1>
                                                                                
                                                                                                                                                                                                                                             root@guerrilla-host-1:~#             =
2023-10-26 17:32:20.423

In [11]:
config = {
    "protocol": "ssh",
    "host": "192.168.121.134",
    "username": "root",
    "password": "vagrant",
    "name": "HOST B"
}
host_b = Session(config)
host_b.connect()

dev = 'eth1'
ip = ['192.168.127.93']
mask = ['255.255.255.0']

# set networks
host_b.run(f'ip addr flush dev {dev}')
host_b.run(f'echo "source-directory /etc/network/interfaces.d" > /etc/network/interfaces')
config = f"auto {dev}\niface {dev} inet static\naddress {ip[0]}\nnetmask {mask[0]}"

for i in range(1, len(ip)):
        host_b.run(f'echo "up ip addr add {ip[i]}/{mask[i]} dev {dev}" >> /etc/network/interfaces.d/{dev}')

2023-10-26 17:32:20.529 | INFO     | guerrilla.connection.ssh:__post_init__:23 - Created HOST B's SSH session to 192.168.121.134:22
2023-10-26 17:32:20.530 | INFO     | guerrilla.connection.ssh:disconnect:44 - Closed connection to 192.168.121.134:22
2023-10-26 17:32:20.800 | SUCCESS  | guerrilla.connection.ssh:connect:35 - Connected to HOST B 192.168.121.134:22
2023-10-26 17:32:20.951 | DEBUG    | guerrilla.connection.ssh:run:78 - From HOST B         sent command: ip addr flush dev eth1
2023-10-26 17:32:21.052 | DEBUG    | guerrilla.connection.ssh:run:96 - From HOST B     received  output: ip addr flush dev eth1>
                                                                                
                                                                                                                                                                                                                                             root@guerrilla-host-2:~#             =
2023-10-26 17:32:21.

In [12]:
dut_iface_ip = '192.168.128.254'
dut_iface_vlan = '2'
dut_iface_port = '8'
dut_iface_mask = '255.255.255.0'

device.run('configure')
device.run(f'vlan create {dut_iface_vlan}')

device.run(f'interface ethernet 1/{dut_iface_port}') # config-if
device.run(f'switchport access vlan {dut_iface_vlan}') 

device.run('exit') # return config
device.run('interface wan') # config-if wan
device.run(f'bind vlan {dut_iface_vlan}')
device.run(f'ip address static {dut_iface_ip} {dut_iface_mask}')

device.run('exit') # return config
device.run('exit') # return main

ret = device.run('show interfaces wan')
assert dut_iface_ip in ret, f'WAN interface IP is not {dut_iface_ip}'

2023-10-26 17:32:21.157 | DEBUG    | guerrilla.connection.ssh:run:78 - From EDR-G9010      sent command: configure
2023-10-26 17:32:21.258 | DEBUG    | guerrilla.connection.ssh:run:96 - From EDR-G9010  received  output: configure
Firewall/VPN Router 00000(config)#
2023-10-26 17:32:21.259 | DEBUG    | guerrilla.connection.ssh:run:78 - From EDR-G9010      sent command: vlan create 2
2023-10-26 17:32:21.359 | DEBUG    | guerrilla.connection.ssh:run:96 - From EDR-G9010  received  output: vlan create 2
% create vlan id:2
2023-10-26 17:32:21.360 | DEBUG    | guerrilla.connection.ssh:run:78 - From EDR-G9010      sent command: interface ethernet 1/8
2023-10-26 17:32:21.460 | DEBUG    | guerrilla.connection.ssh:run:96 - From EDR-G9010  received  output: Firewall/VPN Router 00000(config)# interface ethernet 1/8
Firewall/VPN Router 00000(config-if)#
2023-10-26 17:32:21.461 | DEBUG    | guerrilla.connection.ssh:run:78 - From EDR-G9010      sent command: switchport access vlan 2
2023-10-26 17:32:21

In [ ]:
dut_iface_ip = '192.168.127.254'
dut_iface_vlan = '1'
dut_iface_port = '1'
dut_iface_mask = '255.255.255.0'

device.run('configure')
device.run(f'interface ethernet 1/{dut_iface_port}')
device.run(f'switchport access vlan {dut_iface_vlan}')

device.run('exit') # return config
device.run('interface lan')
device.run(f'bind vlan {dut_iface_vlan}')
device.run(f'ip address static {dut_iface_ip} {dut_iface_mask}')

device.run('exit') # return config
device.run('exit') # return main

ret = device.run('show interface lan')
assert dut_iface_ip in ret, f'LAN interface IP is not {dut_iface_ip}'

In [ ]:
# IP Address
host_testbed_ip = '192.168.128.10'


# Subnet Mask
host_testbed_mask = '255.255.255.0'

# Network name (subnet)
host_testbed_subnet = str(
    ipaddress.ip_network(f'{host_testbed_ip}/{host_testbed_mask}',
                            strict=False)).split('/')[0]

# Gateway IP
host_testbed_gw = '192.168.128.254'

# context.host_info[host]["testbed"]["cur_host"] = host_testbed_ip
# context.host_info[host]["testbed"]["cur_subnet"] = host_testbed_subnet
# context.host_info[host]["testbed"]["cur_gw"] = host_testbed_gw

# set network
dev = 'eth1'
config=f"auto {dev}\niface {dev} inet static\naddress {host_testbed_ip}\nnetmask {host_testbed_mask}"
host_a.run(f'ip addr flush {dev}')
host_a.run('echo "source-directory /etc/network/interfaces.d" > /etc/network/interfaces')
host_a.run(f'echo "{config}" > /etc/network/interfaces.d/{dev}')

host_a.run(f"ifdown {dev}")
host_a.run(f"ifup {dev}")

In [ ]:
# IP Address
host_testbed_ip = '192.168.127.11'

# Subnet Mask
host_testbed_mask = '255.255.255.0'

# Network name (subnet)
host_testbed_subnet = str(
    ipaddress.ip_network(f'{host_testbed_ip}/{host_testbed_mask}',
                            strict=False)).split('/')[0]

# Gateway IP
host_testbed_gw = '192.168.127.254'

# context.host_info[host]["testbed"]["cur_host"] = host_testbed_ip
# context.host_info[host]["testbed"]["cur_subnet"] = host_testbed_subnet
# context.host_info[host]["testbed"]["cur_gw"] = host_testbed_gw

# set network
dev = 'eth1'
config=f"auto {dev}\niface {dev} inet static\naddress {host_testbed_ip}\nnetmask {host_testbed_mask}"
host_b.run(f'ip addr flush {dev}')
host_b.run('echo "source-directory /etc/network/interfaces.d" > /etc/network/interfaces')
host_b.run(f'echo "{config}" > /etc/network/interfaces.d/{dev}')

host_b.run(f"ifdown {dev}")
host_b.run(f"ifup {dev}")

In [ ]:
dev = 'eth1'
dip = '192.168.127.0' # host b subnet
mask = '255.255.255.0' # host b mask
gw = '192.168.128.254' # host a gw


host_a.run(f'ip addr flush {dev}')
host_a.run(f'echo "post-up ip route add {dip}/{mask} via {gw}" >> /etc/network/interfaces.d/{dev}')

host_a.run(f"ifdown {dev}")
host_a.run(f"ifup {dev}")

In [ ]:
dev = 'eth1'
dip = '192.168.128.0' # host a subnet
mask = '255.255.255.0' # host a mask
gw = '192.168.127.254' # host b gw

host_b.run(f'ip addr flush {dev}')
host_b.run(f'echo "post-up ip route add {dip}/{mask} via {gw}" >> /etc/network/interfaces.d/{dev}')

host_b.run(f"ifdown {dev}")
host_b.run(f"ifup {dev}")

# show dut wan ip
device.run('show interfaces wan')

# show host a ip
host_a.run('ip addr show eth1')

# show host b ip
host_b.run('ip addr show eth1')

In [ ]:
ret = host_a.run('ping -c 3 192.168.127.11', is_shell=True)
assert '3 received' in ret, f'HOST_A cannot ping HOST_B'
logger.success(f'HOST_A can ping HOST_B')

In [ ]:
ret = host_b.run('ping -c 3 192.168.128.10', is_shell=True)
assert '3 received' in ret, f'HOST_B cannot ping HOST_A'
logger.success(f'HOST_B can ping HOST_A')
